In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import re

In [2]:
eps = []
titles = []
year = []
content = []

In [3]:
# parses transcript links
class Lore(scrapy.Spider):
    name = 'Lore'
    
    def start_requests(self):
        url = 'https://loretranscripts.tumblr.com/page/2'
        # print(urls) # Urls are correct
        yield scrapy.Request(url=url, callback=self.parse_episode)
        
    def parse_episode(self, response):
        transcript_links = response.css(' div.entry > ul >li > a::attr(href)').extract() 
        # print([l for l in transcript_links])
        for link in transcript_links:
            yield response.follow(url=link, 
                                  callback =self.parse_transcript)
    
    def parse_transcript(self, response):        
        # Title - done, clean
        title = response.css(' div.entry > h1::text').extract()[0].split('Lore Episode ')[1]
        
        ep_number = re.findall(r'\d+', title)[0] # Not ideal re, but works
        eps.append(ep_number)
        
        ep_year = re.findall(r'\d+', title)[-1] # Not ideal re, but it works
        year.append(ep_year)
        
        # ep_title = # NEED REGEX REFRESHER
        # titles.append(ep_title)
        
        
        text = response.css(' div.entry > p::text').extract()
        content.append(text)
        

In [4]:
process = CrawlerProcess()
process.crawl(Lore)
process.start()

2021-03-04 09:54:53 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-03-04 09:54:53 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.18362-SP0
2021-03-04 09:54:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-03-04 09:54:53 [scrapy.crawler] INFO: Overridden settings:
{}
2021-03-04 09:54:53 [scrapy.extensions.telnet] INFO: Telnet Password: 3897220a95760c9b
2021-03-04 09:54:53 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-03-04 09:54:53 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 's

In [5]:
print(eps)
print(year)

['14', '13', '9', '7', '18', '5', '130', '32', '26', '24', '1', '19', '3', '30']
['2015', '2015', '2015', '2015', '2015', '2015', '2019', '2016', '2016', '2015', '2015', '2015', '2015', '2016']


In [6]:
import pandas as pd

# Add info row by row with each episode number as the index
# lore_data = pd.DataFrame( columns = ['Epsiode','Title','Year','Content'])

ValueError: 3 columns passed, passed data had 14 columns